In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import torch
import torch.autograd
from torch.autograd import Variable
import numpy as np
from scipy.io import loadmat
from scipy.special import sph_harm
import math
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from time import time
from mpl_toolkits.mplot3d import axes3d, Axes3D 
from torch.autograd.gradcheck import zero_gradients

In [2]:
# torch.device("cuda")
torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [3]:
no_of_ver = 53215

In [4]:
fileName='Dataset/Coarse_Dataset/Exp_Pca.bin'
with open(fileName, mode='rb') as file: # b is important -> binary
#     fileContent = file.read()
    dim_exp = np.fromfile(file, dtype=np.int32, count=1)
    mu_exp = np.zeros(no_of_ver*3)
    base_exp = np.zeros((no_of_ver*3*dim_exp[0]), dtype=float)
    mu_exp = np.fromfile(file, dtype=np.float32, count=3*no_of_ver)
    base_exp = np.fromfile(file, dtype=np.float32, count=3*no_of_ver*dim_exp[0])

In [5]:
A_exp = torch.tensor(np.array(np.resize(base_exp, (no_of_ver*3, dim_exp[0]))), dtype=torch.float32, requires_grad = True)

In [6]:
data = np.loadtxt('Dataset/Coarse_Dataset/std_exp.txt', delimiter=' ')
data = torch.tensor(data[:,np.newaxis], dtype = torch.float32,requires_grad =True )

In [7]:
temp = loadmat('Dataset/3DDFA_Release/Matlab/ModelGeneration/model_info.mat')
temp['tri'].dtype = np.int16
trimIndex = np.array(temp['trimIndex'][:,0], dtype=np.int32)
trim_ind = np.reshape(np.array([3*trimIndex-2,3*trimIndex-1,3*trimIndex])-1,(no_of_ver*3,),'F')
tri_mesh_data = temp['tri'].T#torch.tensor(temp['tri'].T, dtype=torch.long)

In [8]:
def load(path):
    """takes as input the path to a .pts and returns a list of 
    tuples of floats containing the points in in the form:
    [(x_0, y_0, z_0),
     (x_1, y_1, z_1),
     ...
     (x_n, y_n, z_n)]"""
    with open(path) as f:
        rows = [rows.strip() for rows in f]

    """Use the curly braces to find the start and end of the point data""" 
    head = rows.index('{') + 1
    tail = rows.index('}')

    """Select the point data split into coordinates"""
    raw_points = rows[head:tail]
    coords_set = [point.split() for point in raw_points]

    """Convert entries from lists of strings to tuples of floats"""
    points = [tuple([float(point) for point in coords]) for coords in coords_set]
    return points

In [ ]:
lmks_2d = torch.tensor(load('Dataset/300W-Convert/300W-Original/afw/134212_1.pts'))
lmks_3d_ind = np.array(temp['keypoints'], dtype=np.int32)

In [ ]:
print("PyTorch version: ", torch.__version__ )
print("CUDA available: ", torch.cuda.is_available())
print("CUDA version: ", torch.version.cuda)

PyTorch version:  1.1.0
CUDA available:  True
CUDA version:  9.0.176


In [ ]:
morph_model = loadmat('Dataset/PublicMM1/01_MorphableModel.mat')

In [ ]:
shapePCA = morph_model['shapePC']
shapeMU = morph_model['shapeMU']
shapeSTD = morph_model['shapeEV']

texPCA = morph_model['texPC']
texMU = morph_model['texMU']
texSTD = morph_model['texEV']

In [ ]:
p_mu = torch.tensor(shapeMU[trim_ind])
b_mu = torch.tensor(texMU[trim_ind])
A_alb = torch.tensor(texPCA[trim_ind,:100])
A_id = torch.tensor(shapePCA[trim_ind,:100])
std_id = torch.tensor(shapeSTD[:100])
std_alb = torch.tensor(texSTD[:100])
std_exp = data

In [ ]:
def P(l, m, x):
    pmm = 1.0
    if m>0:
        somx2 = torch.sqrt((1.0-x)*(1.0+x))
        fact = 1.0
        for i in torch.arange(m):
            pmm = -fact*pmm*somx2
            fact = fact+2.0
    if l==m :
        return pmm
    pmmp1 = x*(2.0*m+1.0)*pmm
    if (l==m+1):
        return pmmp1
    pll = 0.0
    for ll in torch.arange(m+2, l+1):
        pll = ((2.0*ll-1.0)*x*pmmp1 - (ll+m-1.0)*pmm)/(ll-m)
        pmm = pmmp1
        pmmp1 = pll
    return pll

def factorial(n):
    return torch.prod(torch.arange(1,n+1), dtype=torch.float32)

def K(l, m):
    norm_const = ((2.0*l+1.0)*factorial(l-m))/((4.0*np.pi)*factorial(l+m))
    return torch.sqrt(norm_const)

def SH(m, l, phi, theta):
    sqrt2 = torch.sqrt(torch.tensor(2.0, dtype=torch.float32))
    if m==0:
        return K(l,0)*P(l,m,torch.cos(theta))
    elif m>0:
        return sqrt2*K(l,m)*torch.cos(m*phi)*P(l,m,torch.cos(theta))
    else:
        return sqrt2*K(l,-m)*torch.sin(-m*phi)*P(l,-m,torch.cos(theta))

In [ ]:
def rot_mat(pitch, yaw, roll):
    Ry = torch.tensor([[torch.cos(pitch),0,torch.sin(pitch)],[0,1,0],[-torch.sin(pitch),0,torch.cos(pitch)]], requires_grad = True)
    Rx = torch.tensor([[1,0,0],[0,torch.cos(roll),torch.sin(roll)],[0,-torch.sin(roll),torch.cos(roll)]], requires_grad = True)
    Rz = torch.tensor([[torch.cos(yaw),torch.sin(yaw),0],[-torch.sin(yaw),torch.cos(yaw),0],[0,0,1]], requires_grad = True)
    R = Rz@Ry@Rx
    
    return R

def sh_basis(norm):
#     if torch.is_tensor(n):
#         norm = n.cpu().detach().numpy()
#     else:
#         norm = n
    theta = norm[1] #Polar angle
    phi = norm[0] #Azimuth angle
    sh = torch.zeros((9,), dtype=torch.float32, requires_grad=True)
    count = 0
    for l in torch.arange(3):
        for m in torch.arange(-l,l+1):
#             if m==0:
#                 sh[count]=np.real(sph_harm(m,l,phi,theta))
#             elif m>0:
#                 sh[count]=np.sqrt(2)*np.real(sph_harm(m,l,phi,theta))
#             else:
#                 sh[count]=np.sqrt(2)*np.imag(sph_harm(m,l,phi,theta))
#             count = count+1
                sh[count] = SH(m,l,phi,theta)
                count = count+1
            
    return sh

In [ ]:
def barycentric_weights(p, tri_p):
    #http://blackpawn.com/texts/pointinpoly/
    
    a = tri_p[0,:]
    b = tri_p[1,:]
    c = tri_p[2,:]
    
    v0 = c - a
    v1 = b - a
    v2 = p - a
    
    dot00 = np.dot(v0, v0)
    dot01 = np.dot(v0, v1)
    dot02 = np.dot(v0, v2)
    dot11 = np.dot(v1, v1)
    dot12 = np.dot(v1, v2)
    
    denom = dot00*dot11 - dot01*dot01
    
    if denom == 0:
        u = v = 0
    else:
        u = (dot11*dot02-dot01*dot12)/denom
        v = (dot00*dot12-dot01*dot02)/denom
    
#     A = torch.stack((v0,v1)).t()
#     B = v2[:,None]
    
#     X, LU = torch.solve(B,A)
#     u = X[0,0]
#     v = X[1,0]
        
#     weights = torch.tensor([1-u-v, u, v], dtype=torch.float32, requires_grad = True)
    weights = np.array([1-u-v, u, v])
    return weights

def world_to_image(q_world, h, w):
#     temp = np.array([w/2,h/2-h+1,0])
#     q_image = (q_world + temp)*[1,-1,1]
    q_image = q_world.clone()
    
    q_image[:,0] = q_image[:,0] + w/2
    q_image[:,1] = q_image[:,1] + h/2
    q_image[:,1] = h - q_image[:,1] - 1
    
    return q_image

def rasterize(q, q_depth, tri_mesh_data, h, w):
    depth_info = -math.inf*np.ones((h,w))
#     depth_info = {}
    tri_ind_info = -np.ones((h,w))
    bary_wts_info = torch.zeros((h,w,3), requires_grad = True)
    
#     for i in range(h):
#         for j in range(w):
#             depth_info[(i,j)] = -math.inf
#             bary_wts_info[(i,j)] = 0
    q_n = q.data.cpu().numpy()
    q_depth_n = q_depth.data.cpu().numpy()
    for i in range(len(tri_mesh_data)):
        print('Rasterizing triangle: ', i+1)
        tri_ver_ind = tri_mesh_data[i,:]
        
        umin = max(int(np.ceil(np.min(q_n[tri_ver_ind, 0]))), 0) #torch.min(lmks_2d[:,0])
        umax = min(int(np.floor(np.max(q_n[tri_ver_ind, 0]))), w-1) #torch.max(lmks_2d[:,0])
        
        vmin = max(int(np.ceil(np.min(q_n[tri_ver_ind, 1]))), 0) #torch.min(lmks_2d[:,0])
        vmax = min(int(np.floor(np.max(q_n[tri_ver_ind, 1]))), h-1)
        
        if umax<umin or vmax<vmin:
            continue
        else:
            for u in range(umin, umax+1):
                for v in range(vmin, vmax+1):
                    weights = barycentric_weights([u, v], q_n[tri_ver_ind, :])
                    if (weights<0).all():
                        continue
                    else:
                        depth = np.dot(weights, q_depth_n[tri_ver_ind])
                        if depth > depth_info[v,u]:
                            depth_info[v,u] = depth
                            tri_ind_info[v,u] = i
                            bary_wts_info[v,u,:] = torch.tensor(weights, requires_grad=True)
                            
    return tri_ind_info, bary_wts_info

In [ ]:
def cart2sph(n):
#     phi = np.arctan2(n[1],n[0]) #arctan(y/x)
#     theta = np.arccos(n[2]) #arccos(z)
    if torch.is_tensor(n):
        norm = n.cpu().detach().numpy()
    else:
        norm = n
    phi = np.arctan2(norm[1],norm[0])
    theta = np.arccos(norm[2])
    return torch.tensor([phi, theta], dtype=torch.float32, requires_grad = True)

def calculate_normal(tri_ind_info, tri_mesh_data, centroid, q, h, w):
    normal_xyz = np.zeros((h, w, 3))
    normal_sph = torch.zeros((h, w, 2), requires_grad = True)
    cent = centroid.data.cpu().numpy()
    for i in range(h):
        for j in range(w):
            tri_ver = q[tri_mesh_data[tri_ind_info[i, j]-1, :],:].data.cpu().numpy()
            a = tri_ver[0,:]
            b = tri_ver[1,:]
            c = tri_ver[2,:]
            normal_xyz[i,j,:] = np.cross(a-b, b-c)/np.linalg.norm(np.cross(a-b, b-c))
            if np.dot(np.mean(tri_ver, 0)-cent, normal_xyz[i,j,:])<0:
                normal_xyz[i,j,:] *= -1
            normal_sph[i,j,:] = cart2sph(normal_xyz[i,j,:])
    return normal_sph

In [ ]:
def isPointInTri(point, tri_points):
    ''' Judge whether the point is in the triangle
    Method:
        http://blackpawn.com/texts/pointinpoly/
    Args:
        point: (2,). [u, v] or [x, y] 
        tri_points: (3 vertices, 2 coords). three vertices(2d points) of a triangle. 
    Returns:
        bool: true for in triangle
    '''
    tp = tri_points

    # vectors
    v0 = tp[2,:] - tp[0,:]
    v1 = tp[1,:] - tp[0,:]
    v2 = point - tp[0,:]
#     A = torch.stack((v0,v1)).t()
#     B = v2[:,None]

#     X = torch.inverse(A)@B

#     u = X[0,0]
#     v = X[1,0]
    # dot products
    dot00 = np.dot(v0, v0)
    dot01 = np.dot(v0, v1)
    dot02 = np.dot(v0, v2)
    dot11 = np.dot(v1, v1)
    dot12 = np.dot(v1, v2)

    # barycentric coordinates
    if dot00*dot11 - dot01*dot01 == 0:
        inverDeno = 0
    else:
        inverDeno = 1/(dot00*dot11 - dot01*dot01)

    u = (dot11*dot02 - dot01*dot12)*inverDeno
    v = (dot00*dot12 - dot01*dot02)*inverDeno

    # check if point in triangle
    return (u >= 0) & (v >= 0) & (u + v < 1)

def get_point_weight(point, tri_points):
    ''' Get the weights of the position
    Methods: https://gamedev.stackexchange.com/questions/23743/whats-the-most-efficient-way-to-find-barycentric-coordinates
     -m1.compute the area of the triangles formed by embedding the point P inside the triangle
     -m2.Christer Ericson's book "Real-Time Collision Detection". faster.(used)
    Args:
        point: (2,). [u, v] or [x, y] 
        tri_points: (3 vertices, 2 coords). three vertices(2d points) of a triangle. 
    Returns:
        w0: weight of v0
        w1: weight of v1
        w2: weight of v3
     '''
    tp = tri_points
    # vectors
    v0 = tp[2,:] - tp[0,:]
    v1 = tp[1,:] - tp[0,:]
    v2 = point - tp[0,:]
    
#     A = torch.stack((v0,v1)).t()
#     B = v2[:,None]
    
#     X = torch.inverse(A)@B
#     u = X[0,0]
#     v = X[1,0]
#     dot products
    dot00 = np.dot(v0, v0)
    dot01 = np.dot(v0, v1)
    dot02 = np.dot(v0, v2)
    dot11 = np.dot(v1, v1)
    dot12 = np.dot(v1, v2)

    # barycentric coordinates
    if dot00*dot11 - dot01*dot01 == 0:
        inverDeno = 0
    else:
        inverDeno = 1/(dot00*dot11 - dot01*dot01)

    u = (dot11*dot02 - dot01*dot12)*inverDeno
    v = (dot00*dot12 - dot01*dot02)*inverDeno

   
    w0 = 1 - u - v
    w1 = v
    w2 = u
    return w0,w1,w2

def rasterize_triangles(vertices, triangles, h, w):
    ''' 
    Args:
        vertices: [nver, 3]
        triangles: [ntri, 3]
        h: height
        w: width
    Returns:
        depth_buffer: [h, w] saves the depth, here, the bigger the z, the fronter the point.
        triangle_buffer: [h, w] saves the tri id(-1 for no triangle). 
        barycentric_weight: [h, w, 3] saves corresponding barycentric weight.

    # Each triangle has 3 vertices & Each vertex has 3 coordinates x, y, z.
    # h, w is the size of rendering
    '''
    # initial 
    depth_buffer = torch.zeros([h, w]) - 999999. #+ torch.min(vertices[2,:]) - 999999. # set the initial z to the farest position
    triangle_buffer = torch.zeros([h, w], dtype = torch.int32) - 1  # if tri id = -1, the pixel has no triangle correspondance
    barycentric_weight = torch.zeros([h, w, 3], dtype = torch.float32)  # 
    
#     for i in range(h):
#         for j in range(w):
#             depth_buffer[(i,j)] = -math.inf
#             barycentric_weight[(i,j)] = 0
    ver = vertices.data.cpu().numpy()
    for i in range(triangles.shape[0]):
        print('Rasterzing: ',i+1)
        tri = triangles[i, :] # 3 vertex indices

        # the inner bounding box
        umin = max(int(np.ceil(np.min(ver[tri, 0]))), 0)
        umax = min(int(np.floor(np.max(ver[tri, 0]))), w-1)

        vmin = max(int(np.ceil(np.min(ver[tri, 1]))), 0)
        vmax = min(int(np.floor(np.max(ver[tri, 1]))), h-1)

        if umax<umin or vmax<vmin:
            continue

        for u in range(umin, umax+1):
            for v in range(vmin, vmax+1):
                if not isPointInTri([u, v], ver[tri, :2]): 
                    continue
                w0, w1, w2 = get_point_weight([u, v], ver[tri, :2]) # barycentric weight
                point_depth = w0*vertices[tri[0], 2] + w1*vertices[tri[1], 2] + w2*vertices[tri[2], 2]
                if point_depth > depth_buffer[v, u]:
                    depth_buffer[(v, u)] = point_depth
                    triangle_buffer[v, u] = i
                    barycentric_weight[v, u, :] = torch.tensor([w0, w1, w2], dtype=torch.float32, requires_grad=True)

    return triangle_buffer, barycentric_weight

In [ ]:
def render_color_image(q, albedo, tri_mesh_data, gamma, h, w):
    image = torch.zeros((h,w,3), dtype=torch.float32, requires_grad = True)
    alb = torch.zeros((h,w,3), dtype=torch.float32, requires_grad = True)
    centroid = torch.mean(q,0)
    
    st = time()
#     tri_ind_info, bary_wts_info = rasterize(q[:,:2], q[:,2], tri_mesh_data, h, w)
    tri_ind_info, bary_wts_info = rasterize_triangles(q, tri_mesh_data, h, w)
    print('Rasterization Done!- %f seconds' %(time()-st))
    
    n_sph = calculate_normal(tri_ind_info, tri_mesh_data, centroid, q, h, w)
    
    for i in range(h):
        for j in range(w):
            sh_func = sh_basis(n_sph[i,j,:])
            alb[i,j,:] = albedo[tri_mesh_data[tri_ind_info[i, j]-1, :],:].t()@bary_wts_info[i,j,:]
            image[i,j,:] = alb[i,j,:]*(gamma.t()@sh_func.squeeze())
            
    return image

In [ ]:
I_in = plt.imread('Dataset/300W-Convert/300W-Original/afw/134212_1.jpg')
I_in=torch.tensor(I_in[160:384,704:928,:],dtype = torch.float32, requires_grad = True)

In [ ]:
def compute_jacobian(inputs, output):
    """
    :param inputs: Batch X Size (e.g. Depth X Width X Height)
    :param output: Batch X Classes
    :return: jacobian: Batch X Classes X Size
    """
    assert inputs.requires_grad

    num_classes = output.shape[0]

    jacobian = torch.zeros((num_classes, inputs.shape[0], inputs.shape[1]))
    grad_output = torch.zeros(output.shape)
    if inputs.is_cuda:
        grad_output = grad_output.cuda()
        jacobian = jacobian.cuda()

    for i in range(num_classes):
        zero_gradients(inputs)
        grad_output.zero_()
        grad_output[i] = 1
        output.backward(grad_output, retain_graph=True)
        jacobian[i] = inputs.grad.data

    return jacobian.squeeze()

In [ ]:
h=w=224
chi = torch.rand(312,1,requires_grad = True, dtype=torch.float32)
# t = torch.zeros(3,1, requires_grad = True, dtype=torch.float32)
count = 1
chi_prev = chi
while True:
    print("Iteration No: ", count)
    
    al_id = chi_prev[0:100]
    al_exp = chi_prev[100:179]
    al_alb = chi_prev[179:279]
    [s, pitch, yaw, roll] = chi_prev[279:283,0]
    t = chi_prev[283:285]
    r = chi_prev[285:]
    gamma = torch.reshape(r,(3,9)).t()
    
    
    p = p_mu + torch.matmul(A_id,al_id) + torch.matmul(A_exp,al_exp)
    b = b_mu + torch.matmul(A_alb,al_alb)
    vertex = torch.reshape(p, (no_of_ver, 3))
    albedo = torch.reshape(b, (no_of_ver, 3))
    
    if count == 1:
        s = 150/(torch.max(vertex) - torch.min(vertex))
    
    R = rot_mat(pitch, yaw, roll)
    q_world = s*R@vertex.t() #+ t
    q_world[:2,:] = q_world[:2,:] + t
    q_image = world_to_image(q_world.t(), h, w)
    print('Rendering Image...')
    st = time()
#     albe, bary_wts_info = render_color_image(q_image, albedo, tri_mesh_data, gamma, h, w)
    I_rend = render_color_image(q_image, albedo, tri_mesh_data, gamma, h, w)
    print('Rendering Done! - %f seconds' %(time()-st) )
    w_l = 100
    w_r = 5e-5
    #E=torch.norm(I_rend - I_in)**2 + torch.norm(al_id/std_id)**2 + w_r*torch.norm(al_alb/std_alb)**2 + torch.norm(al_exp/std_exp)**2
    E_con_r = torch.tensor([np.sqrt(1/28241)*torch.norm(I_rend - I_in)], requires_grad = True)
    E_lan_r = np.sqrt(w_l/68)*torch.norm(lmks_2d - q_image[lmks_3d_ind[0,:],:2], dim=1)
    E_reg_r = np.sqrt(w_r)*torch.cat((al_id/std_id,al_alb/std_alb,al_exp/std_exp))

    E = torch.cat((E_con_r,E_lan_r,E_reg_r[:,0]))
    J = compute_jacobian(chi_prev,E)
    chi_next = chi_prev - torch.pinverse(J.t()@J)@J.t()@E.unsqueeze(1)
    increment = torch.norm(chi_next - chi_prev)
    obj_func_loss = torch.norm(E)**2
    chi_prev = chi_next
    count=count+1
    print('Increment: ', increment)
    print('Loss: ', obj_func_loss)
    if obj_func_loss<10:
        break

Iteration No:  1
Rendering Image...
Rasterzing:  1
Rasterzing:  2
Rasterzing:  3
Rasterzing:  4
Rasterzing:  5
Rasterzing:  6
Rasterzing:  7
Rasterzing:  8
Rasterzing:  9
Rasterzing:  10
Rasterzing:  11
Rasterzing:  12
Rasterzing:  13
Rasterzing:  14
Rasterzing:  15
Rasterzing:  16
Rasterzing:  17
Rasterzing:  18
Rasterzing:  19
Rasterzing:  20
Rasterzing:  21
Rasterzing:  22
Rasterzing:  23
Rasterzing:  24
Rasterzing:  25
Rasterzing:  26
Rasterzing:  27
Rasterzing:  28
Rasterzing:  29
Rasterzing:  30
Rasterzing:  31
Rasterzing:  32
Rasterzing:  33
Rasterzing:  34
Rasterzing:  35
Rasterzing:  36
Rasterzing:  37
Rasterzing:  38
Rasterzing:  39
Rasterzing:  40
Rasterzing:  41
Rasterzing:  42
Rasterzing:  43
Rasterzing:  44
Rasterzing:  45
Rasterzing:  46
Rasterzing:  47
Rasterzing:  48
Rasterzing:  49
Rasterzing:  50
Rasterzing:  51
Rasterzing:  52
Rasterzing:  53
Rasterzing:  54
Rasterzing:  55
Rasterzing:  56
Rasterzing:  57
Rasterzing:  58
Rasterzing:  59
Rasterzing:  60
Rasterzing:  

Rasterzing:  692
Rasterzing:  693
Rasterzing:  694
Rasterzing:  695
Rasterzing:  696
Rasterzing:  697
Rasterzing:  698
Rasterzing:  699
Rasterzing:  700
Rasterzing:  701
Rasterzing:  702
Rasterzing:  703
Rasterzing:  704
Rasterzing:  705
Rasterzing:  706
Rasterzing:  707
Rasterzing:  708
Rasterzing:  709
Rasterzing:  710
Rasterzing:  711
Rasterzing:  712
Rasterzing:  713
Rasterzing:  714
Rasterzing:  715
Rasterzing:  716
Rasterzing:  717
Rasterzing:  718
Rasterzing:  719
Rasterzing:  720
Rasterzing:  721
Rasterzing:  722
Rasterzing:  723
Rasterzing:  724
Rasterzing:  725
Rasterzing:  726
Rasterzing:  727
Rasterzing:  728
Rasterzing:  729
Rasterzing:  730
Rasterzing:  731
Rasterzing:  732
Rasterzing:  733
Rasterzing:  734
Rasterzing:  735
Rasterzing:  736
Rasterzing:  737
Rasterzing:  738
Rasterzing:  739
Rasterzing:  740
Rasterzing:  741
Rasterzing:  742
Rasterzing:  743
Rasterzing:  744
Rasterzing:  745
Rasterzing:  746
Rasterzing:  747
Rasterzing:  748
Rasterzing:  749
Rasterzing:  7

Rasterzing:  1351
Rasterzing:  1352
Rasterzing:  1353
Rasterzing:  1354
Rasterzing:  1355
Rasterzing:  1356
Rasterzing:  1357
Rasterzing:  1358
Rasterzing:  1359
Rasterzing:  1360
Rasterzing:  1361
Rasterzing:  1362
Rasterzing:  1363
Rasterzing:  1364
Rasterzing:  1365
Rasterzing:  1366
Rasterzing:  1367
Rasterzing:  1368
Rasterzing:  1369
Rasterzing:  1370
Rasterzing:  1371
Rasterzing:  1372
Rasterzing:  1373
Rasterzing:  1374
Rasterzing:  1375
Rasterzing:  1376
Rasterzing:  1377
Rasterzing:  1378
Rasterzing:  1379
Rasterzing:  1380
Rasterzing:  1381
Rasterzing:  1382
Rasterzing:  1383
Rasterzing:  1384
Rasterzing:  1385
Rasterzing:  1386
Rasterzing:  1387
Rasterzing:  1388
Rasterzing:  1389
Rasterzing:  1390
Rasterzing:  1391
Rasterzing:  1392
Rasterzing:  1393
Rasterzing:  1394
Rasterzing:  1395
Rasterzing:  1396
Rasterzing:  1397
Rasterzing:  1398
Rasterzing:  1399
Rasterzing:  1400
Rasterzing:  1401
Rasterzing:  1402
Rasterzing:  1403
Rasterzing:  1404
Rasterzing:  1405
Rasterzing

Rasterzing:  2048
Rasterzing:  2049
Rasterzing:  2050
Rasterzing:  2051
Rasterzing:  2052
Rasterzing:  2053
Rasterzing:  2054
Rasterzing:  2055
Rasterzing:  2056
Rasterzing:  2057
Rasterzing:  2058
Rasterzing:  2059
Rasterzing:  2060
Rasterzing:  2061
Rasterzing:  2062
Rasterzing:  2063
Rasterzing:  2064
Rasterzing:  2065
Rasterzing:  2066
Rasterzing:  2067
Rasterzing:  2068
Rasterzing:  2069
Rasterzing:  2070
Rasterzing:  2071
Rasterzing:  2072
Rasterzing:  2073
Rasterzing:  2074
Rasterzing:  2075
Rasterzing:  2076
Rasterzing:  2077
Rasterzing:  2078
Rasterzing:  2079
Rasterzing:  2080
Rasterzing:  2081
Rasterzing:  2082
Rasterzing:  2083
Rasterzing:  2084
Rasterzing:  2085
Rasterzing:  2086
Rasterzing:  2087
Rasterzing:  2088
Rasterzing:  2089
Rasterzing:  2090
Rasterzing:  2091
Rasterzing:  2092
Rasterzing:  2093
Rasterzing:  2094
Rasterzing:  2095
Rasterzing:  2096
Rasterzing:  2097
Rasterzing:  2098
Rasterzing:  2099
Rasterzing:  2100
Rasterzing:  2101
Rasterzing:  2102
Rasterzing

Rasterzing:  2672
Rasterzing:  2673
Rasterzing:  2674
Rasterzing:  2675
Rasterzing:  2676
Rasterzing:  2677
Rasterzing:  2678
Rasterzing:  2679
Rasterzing:  2680
Rasterzing:  2681
Rasterzing:  2682
Rasterzing:  2683
Rasterzing:  2684
Rasterzing:  2685
Rasterzing:  2686
Rasterzing:  2687
Rasterzing:  2688
Rasterzing:  2689
Rasterzing:  2690
Rasterzing:  2691
Rasterzing:  2692
Rasterzing:  2693
Rasterzing:  2694
Rasterzing:  2695
Rasterzing:  2696
Rasterzing:  2697
Rasterzing:  2698
Rasterzing:  2699
Rasterzing:  2700
Rasterzing:  2701
Rasterzing:  2702
Rasterzing:  2703
Rasterzing:  2704
Rasterzing:  2705
Rasterzing:  2706
Rasterzing:  2707
Rasterzing:  2708
Rasterzing:  2709
Rasterzing:  2710
Rasterzing:  2711
Rasterzing:  2712
Rasterzing:  2713
Rasterzing:  2714
Rasterzing:  2715
Rasterzing:  2716
Rasterzing:  2717
Rasterzing:  2718
Rasterzing:  2719
Rasterzing:  2720
Rasterzing:  2721
Rasterzing:  2722
Rasterzing:  2723
Rasterzing:  2724
Rasterzing:  2725
Rasterzing:  2726
Rasterzing

...

Rasterzing:  103858
Rasterzing:  103859
Rasterzing:  103860
Rasterzing:  103861
Rasterzing:  103862
Rasterzing:  103863
Rasterzing:  103864
Rasterzing:  103865
Rasterzing:  103866
Rasterzing:  103867
Rasterzing:  103868
Rasterzing:  103869
Rasterzing:  103870
Rasterzing:  103871
Rasterzing:  103872
Rasterzing:  103873
Rasterzing:  103874
Rasterzing:  103875
Rasterzing:  103876
Rasterzing:  103877
Rasterzing:  103878
Rasterzing:  103879
Rasterzing:  103880
Rasterzing:  103881
Rasterzing:  103882
Rasterzing:  103883
Rasterzing:  103884
Rasterzing:  103885
Rasterzing:  103886
Rasterzing:  103887
Rasterzing:  103888
Rasterzing:  103889
Rasterzing:  103890
Rasterzing:  103891
Rasterzing:  103892
Rasterzing:  103893
Rasterzing:  103894
Rasterzing:  103895
Rasterzing:  103896
Rasterzing:  103897
Rasterzing:  103898
Rasterzing:  103899
Rasterzing:  103900
Rasterzing:  103901
Rasterzing:  103902
Rasterzing:  103903
Rasterzing:  103904
Rasterzing:  103905
Rasterzing:  103906
Rasterzing:  103907


Rasterzing:  104273
Rasterzing:  104274
Rasterzing:  104275
Rasterzing:  104276
Rasterzing:  104277
Rasterzing:  104278
Rasterzing:  104279
Rasterzing:  104280
Rasterzing:  104281
Rasterzing:  104282
Rasterzing:  104283
Rasterzing:  104284
Rasterzing:  104285
Rasterzing:  104286
Rasterzing:  104287
Rasterzing:  104288
Rasterzing:  104289
Rasterzing:  104290
Rasterzing:  104291
Rasterzing:  104292
Rasterzing:  104293
Rasterzing:  104294
Rasterzing:  104295
Rasterzing:  104296
Rasterzing:  104297
Rasterzing:  104298
Rasterzing:  104299
Rasterzing:  104300
Rasterzing:  104301
Rasterzing:  104302
Rasterzing:  104303
Rasterzing:  104304
Rasterzing:  104305
Rasterzing:  104306
Rasterzing:  104307
Rasterzing:  104308
Rasterzing:  104309
Rasterzing:  104310
Rasterzing:  104311
Rasterzing:  104312
Rasterzing:  104313
Rasterzing:  104314
Rasterzing:  104315
Rasterzing:  104316
Rasterzing:  104317
Rasterzing:  104318
Rasterzing:  104319
Rasterzing:  104320
Rasterzing:  104321
Rasterzing:  104322


Rasterzing:  104741
Rasterzing:  104742
Rasterzing:  104743
Rasterzing:  104744
Rasterzing:  104745
Rasterzing:  104746
Rasterzing:  104747
Rasterzing:  104748
Rasterzing:  104749
Rasterzing:  104750
Rasterzing:  104751
Rasterzing:  104752
Rasterzing:  104753
Rasterzing:  104754
Rasterzing:  104755
Rasterzing:  104756
Rasterzing:  104757
Rasterzing:  104758
Rasterzing:  104759
Rasterzing:  104760
Rasterzing:  104761
Rasterzing:  104762
Rasterzing:  104763
Rasterzing:  104764
Rasterzing:  104765
Rasterzing:  104766
Rasterzing:  104767
Rasterzing:  104768
Rasterzing:  104769
Rasterzing:  104770
Rasterzing:  104771
Rasterzing:  104772
Rasterzing:  104773
Rasterzing:  104774
Rasterzing:  104775
Rasterzing:  104776
Rasterzing:  104777
Rasterzing:  104778
Rasterzing:  104779
Rasterzing:  104780
Rasterzing:  104781
Rasterzing:  104782
Rasterzing:  104783
Rasterzing:  104784
Rasterzing:  104785
Rasterzing:  104786
Rasterzing:  104787
Rasterzing:  104788
Rasterzing:  104789
Rasterzing:  104790


Rasterzing:  105189
Rasterzing:  105190
Rasterzing:  105191
Rasterzing:  105192
Rasterzing:  105193
Rasterzing:  105194
Rasterzing:  105195
Rasterzing:  105196
Rasterzing:  105197
Rasterzing:  105198
Rasterzing:  105199
Rasterzing:  105200
Rasterzing:  105201
Rasterzing:  105202
Rasterzing:  105203
Rasterzing:  105204
Rasterzing:  105205
Rasterzing:  105206
Rasterzing:  105207
Rasterzing:  105208
Rasterzing:  105209
Rasterzing:  105210
Rasterzing:  105211
Rasterzing:  105212
Rasterzing:  105213
Rasterzing:  105214
Rasterzing:  105215
Rasterzing:  105216
Rasterzing:  105217
Rasterzing:  105218
Rasterzing:  105219
Rasterzing:  105220
Rasterzing:  105221
Rasterzing:  105222
Rasterzing:  105223
Rasterzing:  105224
Rasterzing:  105225
Rasterzing:  105226
Rasterzing:  105227
Rasterzing:  105228
Rasterzing:  105229
Rasterzing:  105230
Rasterzing:  105231
Rasterzing:  105232
Rasterzing:  105233
Rasterzing:  105234
Rasterzing:  105235
Rasterzing:  105236
Rasterzing:  105237
Rasterzing:  105238


Rasterzing:  105624
Rasterzing:  105625
Rasterzing:  105626
Rasterzing:  105627
Rasterzing:  105628
Rasterzing:  105629
Rasterzing:  105630
Rasterzing:  105631
Rasterzing:  105632
Rasterzing:  105633
Rasterzing:  105634
Rasterzing:  105635
Rasterzing:  105636
Rasterzing:  105637
Rasterzing:  105638
Rasterzing:  105639
Rasterzing:  105640
Rasterzing:  105641
Rasterzing:  105642
Rasterzing:  105643
Rasterzing:  105644
Rasterzing:  105645
Rasterzing:  105646
Rasterzing:  105647
Rasterzing:  105648
Rasterzing:  105649
Rasterzing:  105650
Rasterzing:  105651
Rasterzing:  105652
Rasterzing:  105653
Rasterzing:  105654
Rasterzing:  105655
Rasterzing:  105656
Rasterzing:  105657
Rasterzing:  105658
Rasterzing:  105659
Rasterzing:  105660
Rasterzing:  105661
Rasterzing:  105662
Rasterzing:  105663
Rasterzing:  105664
Rasterzing:  105665
Rasterzing:  105666
Rasterzing:  105667
Rasterzing:  105668
Rasterzing:  105669
Rasterzing:  105670
Rasterzing:  105671
Rasterzing:  105672
Rasterzing:  105673


In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(q_world[0,:].cpu().detach(), q_world[1,:].cpu().detach(), q_world[2,:].cpu().detach())

In [ ]:
ax.view_init(-45,45)
fig

In [ ]:
E_con_r